In [7]:
import pygame
import random
import os

pygame.init()

WIDTH = 1200
HEIGHT = 800
FPS = 60
PLAYER_SPEED = 5
BULLET_SPEED = 5
BASE_ALIEN_SPEED = 2
BASE_ALIEN_SPAWN_RATE = 60
HIGHSCORE_FILE = "highscore.txt"

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Drill Baby Drill")
clock = pygame.time.Clock()

try:
    background_image = pygame.image.load(r"C:\Users\user\Pictures\backs.png")
    background_image = pygame.transform.scale(background_image, (WIDTH, HEIGHT))
except Exception as e:
    print(f"Error loading background image: {e}")
    background_image = pygame.Surface((WIDTH, HEIGHT))
    background_image.fill(BLACK)

try:
    player_image = pygame.image.load(r"C:\Users\user\Pictures\Drill.png")
    player_image = pygame.transform.scale(player_image, (120, 150))
except Exception as e:
    print(f"Error loading player image: {e}")
    player_image = pygame.Surface((50, 50))
    player_image.fill(WHITE)

try:
    bullet_image = pygame.image.load(r"C:\Users\user\Pictures\Bit.png")
    bullet_image = pygame.transform.scale(bullet_image, (20, 40))
except Exception as e:
    print(f"Error loading bullet image: {e}")
    bullet_image = pygame.Surface((20, 40))
    bullet_image.fill((255, 0, 255))

try:
    alien_image = pygame.image.load(r"C:\Users\user\Pictures\aliens.png")
    alien_image = pygame.transform.scale(alien_image, (75, 75))
except Exception as e:
    print(f"Error loading alien image: {e}")
    alien_image = pygame.Surface((50, 50))
    alien_image.fill((0, 255, 0))

try:
    alien2_image = pygame.image.load(r"C:\Users\user\Pictures\aliens2.png")
    alien2_image = pygame.transform.scale(alien2_image, (75, 100))
except Exception as e:
    print(f"Error loading alien image: {e}")
    alien2_image = pygame.Surface((50, 50))
    alien2_image.fill((255, 0, 0))

try:
    alien3_image = pygame.image.load(r"C:\Users\user\Pictures\aliens3.png")
    alien3_image = pygame.transform.scale(alien3_image, (100, 100))
except Exception as e:
    print(f"Error loading alien image: {e}")
    alien3_image = pygame.Surface((100, 100))
    alien3_image.fill((0, 0, 255))

all_sprites = pygame.sprite.Group()
bullets = pygame.sprite.Group()
aliens = pygame.sprite.Group()
aliens2 = pygame.sprite.Group()
aliens3 = pygame.sprite.Group()

LEVELS = [
    {"depth_required": 100, "alien_speed": 2, "spawn_rate_modifier": 1.0, "alien3_hits": 1},
    {"depth_required": 200, "alien_speed": 2, "spawn_rate_modifier": 1.0, "alien3_hits": 2},
    {"depth_required": 400, "alien_speed": 3, "spawn_rate_modifier": 1.0, "alien3_hits": 2},
    {"depth_required": 800, "alien_speed": 3, "spawn_rate_modifier": 0.8, "alien3_hits": 3},
    {"depth_required": 1000, "alien_speed": 4, "spawn_rate_modifier": 1.0, "alien3_hits": 3},
    {"depth_required": 2000, "alien_speed": 4, "spawn_rate_modifier": 0.6, "alien3_hits": 4},
    {"depth_required": 4000, "alien_speed": 5, "spawn_rate_modifier": 1.0, "alien3_hits": 4},
    {"depth_required": 8000, "alien_speed": 5, "spawn_rate_modifier": 0.5, "alien3_hits": 5},
    {"depth_required": 10000, "alien_speed": 6, "spawn_rate_modifier": 0.5, "alien3_hits": 5},
    {"depth_required": 20000, "alien_speed": 6, "spawn_rate_modifier": 0.5, "alien3_hits": 6},
]

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = player_image
        self.rect = self.image.get_rect()
        self.rect.x = WIDTH // 2 - 25
        self.rect.y = 156

    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and self.rect.x > 0:
            self.rect.x -= PLAYER_SPEED
        if keys[pygame.K_RIGHT] and self.rect.x < WIDTH - self.rect.width:
            self.rect.x += PLAYER_SPEED

class Bullet(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = bullet_image
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

    def update(self):
        self.rect.y += BULLET_SPEED
        if self.rect.y > HEIGHT:
            self.kill()

class Alien1(pygame.sprite.Sprite):
    def __init__(self, x, level_speed):
        super().__init__()
        self.image = alien_image
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = HEIGHT - 50
        self.points = 10
        self.speed = level_speed
        all_sprites.add(self)

    def update(self):
        self.rect.y -= self.speed
        if self.rect.y < 0:
            self.kill()

class Alien2(pygame.sprite.Sprite):
    def __init__(self, x, level_speed):
        super().__init__()
        self.image = alien2_image
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = HEIGHT - 50
        self.points = 20
        self.speed = level_speed * 3
        all_sprites.add(self)

    def update(self):
        self.rect.y -= self.speed
        if self.rect.y < 0:
            self.kill()

class Alien3(pygame.sprite.Sprite):
    def __init__(self, x, level_speed, hits_required):
        super().__init__()
        self.image = alien3_image
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = HEIGHT - 50
        self.points = 30
        self.speed = level_speed / 2
        self.hits = 0
        self.hits_required = hits_required
        all_sprites.add(self)

    def update(self):
        self.rect.y -= self.speed
        if self.rect.y < 0:
            self.kill()

def load_highscore():
    try:
        if os.path.exists(HIGHSCORE_FILE):
            with open(HIGHSCORE_FILE, "r") as file:
                return int(file.read())
        return 0
    except Exception as e:
        print(f"Error loading highscore: {e}")
        return 0

def save_highscore(score):
    try:
        with open(HIGHSCORE_FILE, "w") as file:
            file.write(str(score))
    except Exception as e:
        print(f"Error saving highscore: {e}")

player = Player()
all_sprites.add(player)
Depth = 0
level = 0
highscore = load_highscore()
font = pygame.font.SysFont(None, 36)
game_over = False

def get_current_level():
    for i, lvl in enumerate(LEVELS):
        if Depth < lvl["depth_required"]:
            return i
    return len(LEVELS) - 1

def spawn_alien():
    current_level = get_current_level()
    level_speed = LEVELS[current_level]["alien_speed"]
    x = random.randint(0, WIDTH - 50)
    aliens.add(Alien1(x, level_speed))

def spawn_alien2():
    current_level = get_current_level()
    level_speed = LEVELS[current_level]["alien_speed"]
    x = random.randint(0, WIDTH - 50)
    aliens2.add(Alien2(x, level_speed))

def spawn_alien3():
    current_level = get_current_level()
    level_speed = LEVELS[current_level]["alien_speed"]
    hits_required = LEVELS[current_level]["alien3_hits"]
    x = random.randint(0, WIDTH - 50)
    aliens3.add(Alien3(x, level_speed, hits_required))

def setup():
    global Depth, game_over, level
    Depth = 0
    level = 0
    game_over = False
    all_sprites.empty()
    bullets.empty()
    aliens.empty()
    aliens2.empty()
    aliens3.empty()
    all_sprites.add(player)
    player.rect.x = WIDTH // 2 - 25
    player.rect.y = 156

running = True
setup()
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE and len(bullets) < 5:
                bullet = Bullet(player.rect.x + player.image.get_width() // 2 - bullet_image.get_width() // 2, player.rect.y + player.image.get_height())
                all_sprites.add(bullet)
                bullets.add(bullet)
            if event.key == pygame.K_r and game_over:
                setup()

    if not game_over:
        current_level = get_current_level()
        if current_level != level:
            level = current_level

        all_sprites.update()

        spawn_rate_modifier = LEVELS[current_level]["spawn_rate_modifier"]
        if random.randint(1, int(BASE_ALIEN_SPAWN_RATE * spawn_rate_modifier)) == 1:
            spawn_alien()
        if random.randint(1, int(BASE_ALIEN_SPAWN_RATE * 2 * spawn_rate_modifier)) == 1:
            spawn_alien2()
        if random.randint(1, int(BASE_ALIEN_SPAWN_RATE * 3 * spawn_rate_modifier)) == 1:
            spawn_alien3()

        for alien_group in (aliens, aliens2, aliens3):
            hits = pygame.sprite.groupcollide(bullets, alien_group, True, False)
            for bullet, alien_list in hits.items():
                for alien in alien_list:
                    if isinstance(alien, Alien3):
                        alien.hits += 1
                        if alien.hits >= alien.hits_required:
                            Depth += alien.points
                            alien.kill()
                    else:
                        Depth += alien.points
                        alien.kill()
            if pygame.sprite.spritecollide(player, alien_group, True):
                game_over = True
                if Depth > highscore:
                    highscore = Depth
                    save_highscore(highscore)

        screen.blit(background_image, (0, 0))
        all_sprites.draw(screen)
        Depth_text = font.render(f"Depth: {Depth}", True, WHITE)
        level_text = font.render(f"Level: {level + 1}", True, WHITE)
        highscore_text = font.render(f"Highscore: {highscore}", True, WHITE)
        screen.blit(Depth_text, (10, 10))
        screen.blit(level_text, (10, 50))
        screen.blit(highscore_text, (10, 90))

        if game_over:
            game_over_text = font.render(f"Game Over! Depth: {Depth} Level: {level + 1}", True, WHITE)
            highscore_display_text = font.render(f"Highscore: {highscore}", True, WHITE)
            restart_text = font.render("Press R to Restart", True, WHITE)
            screen.blit(game_over_text, (WIDTH // 2 - 150, HEIGHT // 2 - 40))
            screen.blit(highscore_display_text, (WIDTH // 2 - 80, HEIGHT // 2))
            screen.blit(restart_text, (WIDTH // 2 - 80, HEIGHT // 2 + 40))

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()